# Import all libraries

*cv2*: OpenCV library, which provides tools for computer vision tasks such as image processing, feature detection, and object recognition.

*numpy*: numerical computing, which is commonly used for data manipulation and array operations.

*tensorflow*: Main library of machine learning for building and training deep learning models.

*tensorflow_datasets*: Provides preprocessed datasets for machine learning tasks.

*matplotlib*: Creating visualizations, which may be used for plotting model performance during training.

*import of Dense, Conv2D, MaxPooling2D, Flatten, Dropout, and BatchNormalization*: These are Keras layers from TensorFlow that can be used to build neural networks for various machine learning tasks, more for image classification.

In [ ]:
import os
import random
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization

## Set hyperparameters, for affect model performance
batch_size(128) - how much images the model will process at a time during each training iteration.
epochs(30) - number of model iterations during training process
Callback functions which used during training process:
* tensorboard_callback - allows you to use TensorBoard, which is a visualization tool provided by TensorFlow, in web UI: http://localhost:6006/
* stop_val_loss - monitors the validation loss and stops the training process if it does not improve for a certain number of epochs(30)
* stop_val_accuracy - monitors the validation accuracy and stops the training process if it does not improve for a certain number of epochs(30)
* lr_schedule_val_loss - monitors the validation loss and reduces the learning rate (lr) when it does not improve for a certain number of epochs(30)

In [ ]:
dataset_name = 'cifar10'
image_directory = 'random_images'
batch_size = 128
n_epochs = 30
tensorboard_callback = TensorBoard(log_dir=f'logs/{dataset_name}')
stop_val_loss = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
stop_val_accuracy = EarlyStopping(monitor='val_acc', mode='min', verbose=1, patience=20)
lr_schedule_val_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)